In [67]:
import datetime
import time
import calcTimeStamp
from datetime import timezone
from datetime import datetime, timedelta

# Calculate the time and date for calculations so far

xy = calcTimeStamp.calcTimeStamp()
start, end = (xy[0], xy[1])
start = str(start)
end = str(end)

In [68]:
import collections
import hashlib
import hmac
import time
from datetime import datetime
import requests
import json
import pprint

parameters = {
  "api-key": "vy8jbrjsxlbwgojepq3vfyfqfywyhvbd", 
  "api-secret": "sdqfm6wdfy9w0pqp2vdka38o6b4vcsvc",
  "station-id": 81211, 
  "end-timestamp": end,
  "start-timestamp": start,
  "t": int(time.time())
}

parameters = collections.OrderedDict(sorted(parameters.items()))

for key in parameters:
    print("Parameter name: \"{}\" has value \"{}\"".format(key, parameters[key]))

apiSecret = parameters["api-secret"];
parameters.pop("api-secret", None);

data = ""
for key in parameters:
    data = data + key + str(parameters[key])

print("Data string to hash is: \"{}\"".format(data))
print('\n')

"""
Calculate the HMAC SHA-256 hash that will be used as the API Signature.
"""
apiSignature = hmac.new(
  apiSecret.encode('utf-8'),
  data.encode('utf-8'),
  hashlib.sha256
).hexdigest()

"""
Let's see what the final API Signature looks like.
"""
print("API Signature is: \"{}\"".format(apiSignature))
print('\n')

# Building the URL to get the station

first_part = ('https://api.weatherlink.com/v2/historic/81211?')
api_key = ('api-key=vy8jbrjsxlbwgojepq3vfyfqfywyhvbd')
add_apisig = ('&api-signature=')
add_t = ('&t='+ str(int(time.time())))
start1 = "&start-timestamp=" + start
end1 = "&end-timestamp=" + end

#
URLfinal = (first_part + api_key + add_t + start1 + end1 + add_apisig + apiSignature)
print(URLfinal)

r =  requests.get(URLfinal)
r.encoding = 'utf-8'
print(r)

r =  requests.get(URLfinal)
data_dir_file = '/Users/jameshayes/davis.json' 
with open(data_dir_file, "w") as fd:   
     json.dump(r.json(), fd)

Parameter name: "api-key" has value "vy8jbrjsxlbwgojepq3vfyfqfywyhvbd"
Parameter name: "api-secret" has value "sdqfm6wdfy9w0pqp2vdka38o6b4vcsvc"
Parameter name: "end-timestamp" has value "1678395676"
Parameter name: "start-timestamp" has value "1678338000"
Parameter name: "station-id" has value "81211"
Parameter name: "t" has value "1678395676"
Data string to hash is: "api-keyvy8jbrjsxlbwgojepq3vfyfqfywyhvbdend-timestamp1678395676start-timestamp1678338000station-id81211t1678395676"


API Signature is: "84a6498bb572d39d8cb501551e1278d0ab64d31194590bf069ed00c7c9bfe499"


https://api.weatherlink.com/v2/historic/81211?api-key=vy8jbrjsxlbwgojepq3vfyfqfywyhvbd&t=1678395676&start-timestamp=1678338000&end-timestamp=1678395676&api-signature=84a6498bb572d39d8cb501551e1278d0ab64d31194590bf069ed00c7c9bfe499
<Response [200]>


In [69]:
import collections
import hashlib
import hmac
import time
from datetime import datetime
import requests
import json
import pprint

with open('/Users/jameshayes/davis.json') as fr:
    davisAPI = json.load(fr)
    
  
a = davisAPI['sensors']    
b = a[1]
c = (b['data'])
cLen = len(c)
print(cLen)

with open('/Users/jameshayes/testout.csv', 'w') as outfile: 
    i = 0
    while i < cLen:
        d = c[i]
        hi_temp = (d['temp_hi'])
        lo_temp = (d['temp_lo'])
        rainfall = (d['rainfall_in'])
        print(f'{hi_temp},{lo_temp},{rainfall}', file = outfile)
        i += 1

192


In [70]:
import pandas as pd

#
# Read in the CSV file for processing in pandas
#

path_name = '/Users/jameshayes/'
file_name = 'testout.csv'
full_file = (f'{path_name}{file_name}')

df = pd.read_csv(full_file, index_col=False,names=['temp_hi', 'temp_lo', 'rainfall'])


In [71]:
pd.set_option('display.max_rows', 1440)
pd.set_option('display.max_columns', 35)
pd.set_option('display.width', 1500)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)

max_temp  = (df.sort_values(by='temp_hi', ascending=False))
max_T = max_temp.iloc[:1]
maxT = max_T['temp_hi'].values[0]
maxT = round(maxT)

min_temp  = (df.sort_values(by='temp_lo', ascending=True))
min_T = min_temp.iloc[:1]
minT = min_T['temp_lo'].values[0]
minT = round(minT)
minT = str(minT)
minT = minT.strip()

tot_rain = df['rainfall'].sum()
totR = round(tot_rain)

new_line = '\n'

# write the data to a csv file with an html suffix

with open('/Users/jameshayes/Sites/HiLoRain.txt', 'w') as outfile:
    print(f'The high so far today is: {maxT}{new_line}The low so far today is: {minT}{new_line}Rainfall so far today is: {rainfall}',file = outfile)